In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
from functools import partial
import timm.models.vision_transformer
# %matplotlib inline
# Set up TensorBoard
# writer = SummaryWriter('runs/cifar100_resnet18')
writer_1 = SummaryWriter('runs/cifar100_transformer')
class config:
    seed = 42
    epoch = 100
    best = 0
    learning_rate = 5e-4
    batch_size = 100
    num_workers = 2
    num_classes = 100
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


trainset = dataset.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=config.batch_size,
                                          shuffle=True, num_workers=2)

# Load the test set
testset = dataset.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)

testloader = DataLoader(testset, batch_size=config.batch_size,
                                         shuffle=False, num_workers=2)

# The classes
classes = trainset.classes


100%|██████████| 169001437/169001437 [00:04<00:00, 41639157.30it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
model = torchvision.models.vision_transformer.VisionTransformer(
        image_size=32,
        patch_size=2,
        num_layers=5,
        num_heads=8,
        hidden_dim=256,
        mlp_dim=300,
        dropout=0.01,
        attention_dropout = 0.01,
        num_classes = 100,
).to(config.device)

# class VisionTransformer(timm.models.vision_transformer.VisionTransformer):
#     """ Vision Transformer with support for global average pooling
#     """
#     def __init__(self, global_pool=False, **kwargs):
#         super(VisionTransformer, self).__init__(**kwargs)

#         self.global_pool = global_pool
#         if self.global_pool:
#             norm_layer = kwargs['norm_layer']
#             embed_dim = kwargs['embed_dim']
#             self.fc_norm = norm_layer(embed_dim)

#             del self.norm  # remove the original norm

#     def forward_features(self, x):
#         B = x.shape[0]
#         x = self.patch_embed(x)

#         cls_tokens = self.cls_token.expand(B, -1, -1)
#         x = torch.cat((cls_tokens, x), dim=1)
#         x = x + self.pos_embed
#         x = self.pos_drop(x)

#         for blk in self.blocks:
#             x = blk(x)

#         if self.global_pool:
#             x = x[:, 1:, :].mean(dim=1)  # global pool without cls token
#             outcome = self.fc_norm(x)
#         else:
#             x = self.norm(x)
#             outcome = x[:, 0]

#         return outcome


# def vit_tiny_patch2(**kwargs):
#     model = VisionTransformer(
#         img_size=32, patch_size=2, embed_dim=156, depth=6, num_heads=4, mlp_ratio=4,
#         qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
#     return model

# model = vit_tiny_patch2().to(config.device)
# # Initialize the model and move the model to the device

# total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print('Total parameters:', total_params)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=config.learning_rate,weight_decay=2*1e-4,betas=(0.9,0.998))

print(f'total params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

def train(epoch):
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(config.device))
        loss = criterion(outputs, labels.to(config.device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i%240 ==0:
            print(f'loss: {running_loss}')
        # log the running loss
        writer_1.add_scalar('training loss',
                        running_loss,
                        epoch * len(trainloader) + i)
        
        running_loss = 0.0

    print(f"Finished Training Epoch {epoch}")


def test(epoch):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images.to(config.device))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted.to('cpu') == labels).sum().item()
        acc = correct / total
        # log the test accuracy
        writer_1.add_scalar('test accuracy',
                        acc,
                        epoch)
    if acc>config.best:
        config.best=acc
        torch.save(model,'best.pth')

    print(f'best test acc: {config.best*100}')
    print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}")


total params: 2187328


In [5]:
for epoch in tqdm(range(config.epoch)):
    print(epoch+1)
    train(epoch)
    test(epoch)

print('Finished Training')

# After all the logging is done, close the SummaryWriter
writer_1.close()


  0%|          | 0/100 [00:00<?, ?it/s]

1


0it [00:00, ?it/s]

loss: 4.6051716804504395
loss: 3.628552198410034
loss: 3.694643259048462
Finished Training Epoch 0
best test acc: 17.419999999999998
Accuracy of the network on the 10000 test images: 17.42
2


0it [00:00, ?it/s]

...

loss: 0.12338407337665558
loss: 0.11647258698940277
loss: 0.15750402212142944
Finished Training Epoch 97
best test acc: 43.71
Accuracy of the network on the 10000 test images: 42.02
99


0it [00:00, ?it/s]

loss: 0.10521974414587021
loss: 0.1052030548453331
loss: 0.16502177715301514
Finished Training Epoch 98
best test acc: 43.71
Accuracy of the network on the 10000 test images: 42.4
100


0it [00:00, ?it/s]

loss: 0.06544265896081924
loss: 0.12937915325164795
loss: 0.18583349883556366
Finished Training Epoch 99
best test acc: 43.71
Accuracy of the network on the 10000 test images: 42.45
Finished Training
